In [7]:
%pip install ipykernel pandas h3 duckdb

Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd
import h3
import duckdb
import numpy as np

In [9]:
trips = pd.read_csv("Trip_Data.csv")
RES = 9  # adjust resolution based on city block scale
trips['pickup_h3'] = trips.apply(lambda r: h3.latlng_to_cell(r['Pick Up Latitude'], r['Pick Up Longitude'], RES), axis=1)
trips['dropoff_h3'] = trips.apply(lambda r: h3.latlng_to_cell(r['Drop Off Latitude'], r['Drop Off Longitude'], RES), axis=1)

In [10]:
venue_counts_dropoff = trips.groupby('dropoff_h3')['Drop Off Address'] \
    .agg(lambda x: x.value_counts().idxmax()) \
    .reset_index() \
    .rename(columns={'Drop Off Address':'canonical_dropoff'})

# Pickup canonical
venue_counts_pickup = trips.groupby('pickup_h3')['Pick Up Address'] \
    .agg(lambda x: x.value_counts().idxmax()) \
    .reset_index() \
    .rename(columns={'Pick Up Address':'canonical_pickup'})

In [11]:
trips = trips.merge(venue_counts_dropoff, on='dropoff_h3', how='left')
trips = trips.merge(venue_counts_pickup, on='pickup_h3', how='left')

# Save augmented trips CSV
trips.to_csv("trips_augmented.csv", index=False)

In [12]:
pickup_venues = trips.groupby('pickup_h3')['Pick Up Address'].unique().reset_index()
pickup_venues = pickup_venues.rename(columns={'pickup_h3':'h3', 'Pick Up Address':'pickup_addresses'})

# Drop-off addresses
dropoff_venues = trips.groupby('dropoff_h3')['Drop Off Address'].unique().reset_index()
dropoff_venues = dropoff_venues.rename(columns={'dropoff_h3':'h3', 'Drop Off Address':'dropoff_addresses'})

# Merge pickups and drop-offs into one table
all_venues = pickup_venues.merge(dropoff_venues, on='h3', how='outer')


In [13]:
all_venues['all_addresses'] = all_venues.apply(
    lambda row: list(set(
        list(row['pickup_addresses']) if isinstance(row['pickup_addresses'], (list, np.ndarray)) else []
        + list(row['dropoff_addresses']) if isinstance(row['dropoff_addresses'], (list, np.ndarray)) else []
    )), axis=1
)

In [14]:
all_venues = all_venues[['h3', 'all_addresses']]

In [15]:
con = duckdb.connect('fetii.duckdb')

# Trips table
con.register('trips_df', trips)
con.execute("CREATE OR REPLACE TABLE trips AS SELECT * FROM trips_df;")

# Venues table
con.register('venues_df', all_venues)
con.execute("CREATE OR REPLACE TABLE venues AS SELECT * FROM venues_df;")

# Close connection
con.close()

In [16]:
con = duckdb.connect('fetii.duckdb')

columns_trips = con.execute("PRAGMA table_info('trips');").fetchdf()
columns_venues = con.execute("PRAGMA table_info('venues');").fetchdf()

print("Trips table columns:\n", columns_trips)
print("\nVenues table columns:\n", columns_venues)

Trips table columns:
     cid                name     type  notnull dflt_value     pk
0     0             Trip ID   BIGINT    False       None  False
1     1     Booking User ID   BIGINT    False       None  False
2     2    Pick Up Latitude   DOUBLE    False       None  False
3     3   Pick Up Longitude   DOUBLE    False       None  False
4     4   Drop Off Latitude   DOUBLE    False       None  False
5     5  Drop Off Longitude   DOUBLE    False       None  False
6     6     Pick Up Address  VARCHAR    False       None  False
7     7    Drop Off Address  VARCHAR    False       None  False
8     8  Trip Date and Time  VARCHAR    False       None  False
9     9    Total Passengers   BIGINT    False       None  False
10   10           pickup_h3  VARCHAR    False       None  False
11   11          dropoff_h3  VARCHAR    False       None  False
12   12   canonical_dropoff  VARCHAR    False       None  False
13   13    canonical_pickup  VARCHAR    False       None  False

Venues table colu

In [17]:
# checking h3 matching

con = duckdb.connect('fetii.duckdb')
df_check = con.execute("""SELECT
    t."Drop Off Address" AS original_dropoff,
    t.canonical_dropoff AS trips_canonical,
    v.all_addresses AS venues_all_addresses,
    'MATCH' AS check  -- since it’s in the same H3
FROM trips t
JOIN venues v
    ON t.dropoff_h3 = v.h3
LIMIT 5;
""").df()


print(df_check)

con.close()

                                    original_dropoff  \
0  Robert L. Patton Building (RLP), East 23rd Str...   
1  Cabo Bob's Burritos, Rio Grande Street, Austin...   
2                 601 Brushy Street, Austin, TX, USA   
3  The Aquarium on 6th, East 6th Street, Austin, ...   
4     Coconut Club, Colorado Street, Austin, TX, USA   

                                     trips_canonical  \
0  Robert L. Patton Building (RLP), East 23rd Str...   
1  El Patio Restaurant, Guadalupe Street, Austin,...   
2        Mala Vida, East 6th Street, Austin, TX, USA   
3  The Aquarium on 6th, East 6th Street, Austin, ...   
4     Coconut Club, Colorado Street, Austin, TX, USA   

                                venues_all_addresses  check  
0  [University Campus, San Jacinto Blvd, Austin, ...  MATCH  
1  [North University, Guadalupe St, Austin, Unite...  MATCH  
2  [Sixth Street, E 6th St, Austin, United States...  MATCH  
3  [Sixth Street, Red River St, Austin, United St...  MATCH  
4  [Congress Ave